In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
import pandas as pd
import os
import math
import keras
import nibabel as nib
%matplotlib inline

In [ ]:
#File_name組成由 'Patien_index'(6位數)_'Study_index'(2位數)_'Series_ID'(2位數)_'Key_slice_index'(3位數)
"%06d" % df.iloc[0]['Patient_index'] + "_" + "%02d" % df.iloc[0]['Study_index'] + "_" + \
"%02d" % df.iloc[0]['Series_ID'] + "_" + "%03d" % df.iloc[0]['Key_slice_index'] + ".png"

In [ ]:
#將各個nifti檔另外存成2D圖---ok
df = pd.read_csv('DL_info.csv')
niftiPath = 'C:/Users/Big data/Desktop/Images_nift/'
#核對 file_name 在資料夾內的順序
Images_png_Dir = 'E:/Anaconda3/Test_Image/Images_png/'
Images_png_list = os.listdir(Images_png_Dir)

for j in range(len(df)):
    folder_Name = "%06d" % df.iloc[j]['Patient_index'] + "_" + "%02d" % df.iloc[j]['Study_index'] + "_" + \
    "%02d" % df.iloc[j]['Series_ID']
    
    nifti_Name = "%06d" % df.iloc[j]['Patient_index'] + "_" + "%02d" % df.iloc[j]['Study_index'] + "_" + \
    "%02d" % df.iloc[j]['Series_ID'] + "_" + Slice_range_start + "-" + Slice_range_end + ".nii.gz"
    print('nifti_Name=', nifti_Name)

    Slice_range_start = df.iloc[j]['Slice_range'].split(', ')[0].zfill(3)
    Slice_range_end = df.iloc[j]['Slice_range'].split(', ')[1].zfill(3)
    
    #讀取nifti檔案
    img = nib.load(niftiPath + nifti_Name)
    img_data = img.get_fdata()
    print('shape=', img_data.shape)
    #print(img_data)  #numpy.ndarray
    #照片對比須調整plt.imsave的vmin, vmax(需參照 DICOM_windows value)
    window_start = pd.to_numeric(df.iloc[j]['DICOM_windows'].split(', ')[0]).astype(np.int64)
    window_end = pd.to_numeric(df.iloc[j]['DICOM_windows'].split(', ')[1]).astype(np.int64)
    #確認 folder_Name(ex:000001_01_01)是否跟 Images_png 符合
    #比對 Key_slice_index在nifti的位置, 並取 2D 圖
    try:
        if folder_Name in Images_png_list:
            #print('folder_Name=', folder_Name)
            file_list = os.listdir(Images_png_Dir + folder_Name + '/')
            #print('file_list=', file_list)
            file_name = "%03d" % df.iloc[j]['Key_slice_index'] + ".png"
            #print('file_name=', file_name)  #str
            file_seq = file_list.index(file_name)
            #print('file_seq=', file_seq)
            #確認folder中的順序後, 以File_name為主, 取出slice
            slice = img_data[:, :, file_seq]
            #print('slice=', slice)
            #讀取特定slice並另存成灰階圖
            plt.imsave('E:/Anaconda3/Test_Image/CT_key_slice_image/' + df.iloc[j]['File_name'], slice, cmap='Greys_r', 
                       vmin=window_start, vmax=window_end)
        else:
            print('Not found')
    except:
        continue

In [ ]:
#框出病徵(Bounding_Boxes)
for k in range(len(df)):    
    df = pd.read_csv('DL_info.csv')
    Xmin = pd.to_numeric(df.iloc[k]['Bounding_boxes'].split(', ')[0]).astype(np.int64)
    Ymin = pd.to_numeric(df.iloc[k]['Bounding_boxes'].split(', ')[1]).astype(np.int64)
    Xmax = pd.to_numeric(df.iloc[k]['Bounding_boxes'].split(', ')[2]).astype(np.int64)
    Ymax = pd.to_numeric(df.iloc[k]['Bounding_boxes'].split(', ')[3]).astype(np.int64)
    print('Xmin={}, Ymin={}, Xmax={}, Ymax={}'.format(Xmin, Ymin, Xmax, Ymax))
    #讀取照片檔案
    CT_key_slice_Dir = 'E:/Anaconda3/Test_Image/CT_key_slice_image/'
    img = cv2.imread(CT_key_slice_Dir + df.iloc[k]['File_name'])
    #框選圖片
    #cv2.rectangle(img, (x,y), (x+w , y+h), (255, 0, 0), 2)
    rec_img = cv2.rectangle(img, (Xmin, Ymin), (Xmax, Ymax), (0, 255, 0), 2)
    '''
    cv2.imshow(df.iloc[k]['File_name'], rec_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    '''
    #將框好的全圖檔, 全圖檔存在另外的資料夾
    cv2.imwrite('./CT_key_slice_image_rec/' + df.iloc[k]['File_name'], rec_img)

In [ ]:
#將照片類別分類...ok
#train: Train_Val_Test = 1, test: Train_Val_Test = 2
df = pd.read_csv('DL_info.csv')
key_slice_image_Dir = 'E:/Anaconda3/Test_Image/CT_key_slice_image/'
key_slice_image_class_Dir = 'E:/Anaconda3/Test_Image/CT_key_slice_image_class/'
for i in range(len(df)):
    key_slice_img = cv2.imread(key_slice_image_Dir + df.iloc[i]['File_name'])
    Train_Val_Test_type = df.iloc[i]['Train_Val_Test']
    if Train_Val_Test_type == 1:
        cv2.imwrite( key_slice_image_class_Dir + 'train_image/' + df.iloc[i]['File_name'], key_slice_img)
    elif Train_Val_Test_type == 2:
        cv2.imwrite( key_slice_image_class_Dir + 'test_image/' + df.iloc[i]['File_name'], key_slice_img)
    else:
        cv2.imwrite( key_slice_image_class_Dir + 'other/' + df.iloc[i]['File_name'], key_slice_img)

In [ ]:
#篩選 Train_Val_Test=1, 顯示 File_name, Bounding_boxes
df = pd.read_csv('DL_info.csv')
train_df = df.loc[df['Train_Val_Test']==1, ['File_name', 'Bounding_boxes', 'Coarse_lesion_type', 'Train_Val_Test']]
#將Bounding_boxes 4個數值各別取出 (Xmin, Ymin, Xmax, Ymax)後, 再結合Train_Val_Test == 2, 並另存csv檔
count = len(open('DL_info.csv','r').readlines())
for i in range(count):
    Xmin = pd.to_numeric(train_df.iloc[i]['Bounding_boxes'].split(', ')[0]).astype(np.int64)
    Ymin = pd.to_numeric(train_df.iloc[i]['Bounding_boxes'].split(', ')[1]).astype(np.int64)
    Xmax = pd.to_numeric(train_df.iloc[i]['Bounding_boxes'].split(', ')[2]).astype(np.int64)
    Ymax = pd.to_numeric(train_df.iloc[i]['Bounding_boxes'].split(', ')[3]).astype(np.int64)
    #print('Xmin={}, Ymin={}, Xmax={}, Ymax={}'.format(Xmin, Ymin, Xmax, Ymax))
    train_data = ('train_image/' + train_df.iloc[i]['File_name']+','+str(Xmin)+','+str(Ymin)+ ','+str(Xmax)+','+str(Ymax)+','
                  +str(train_df.iloc[i]['Coarse_lesion_type']))
    #print(type(train_data))
    print(train_data)
    #手動將print出的結果複製, 並存成csv檔

In [ ]:
#篩選 Train_Val_Test=2, 顯示 File_name, Bounding_boxes
df = pd.read_csv('DL_info.csv')
val_df = df.loc[df['Train_Val_Test']==2, ['File_name', 'Bounding_boxes', 'Coarse_lesion_type', 'Train_Val_Test']]
#將Bounding_boxes 4個數值各別取出 (Xmin, Ymin, Xmax, Ymax)後, 再結合Train_Val_Test == 2, 並另存csv檔
count = len(open('DL_info.csv','r').readlines())
for i in range(count):
    Xmin = pd.to_numeric(val_df.iloc[i]['Bounding_boxes'].split(', ')[0]).astype(np.int64)
    Ymin = pd.to_numeric(val_df.iloc[i]['Bounding_boxes'].split(', ')[1]).astype(np.int64)
    Xmax = pd.to_numeric(val_df.iloc[i]['Bounding_boxes'].split(', ')[2]).astype(np.int64)
    Ymax = pd.to_numeric(val_df.iloc[i]['Bounding_boxes'].split(', ')[3]).astype(np.int64)
    #print('Xmin={}, Ymin={}, Xmax={}, Ymax={}'.format(Xmin, Ymin, Xmax, Ymax))
    val_data = ('validation_image/' + val_df.iloc[i]['File_name']+','+str(Xmin)+','+str(Ymin)+ ','+str(Xmax)+','+str(Ymax)+','+str(val_df.iloc[i]['Coarse_lesion_type']))
    #print(type(val_data))
    print(val_data)
    #手動將print出的結果複製, 並存成csv檔

In [ ]:
#篩選 Train_Val_Test=3, 顯示 File_name, Bounding_boxes
df = pd.read_csv('DL_info.csv')
test_df = df.loc[df['Train_Val_Test']==3, ['File_name', 'Bounding_boxes', 'Coarse_lesion_type', 'Train_Val_Test']]
#將Bounding_boxes 4個數值各別取出 (Xmin, Ymin, Xmax, Ymax)後, 再結合Train_Val_Test == 3, 並另存csv檔
count = len(open('DL_info.csv','r').readlines())
for i in range(count):
    Xmin = pd.to_numeric(test_df.iloc[i]['Bounding_boxes'].split(', ')[0]).astype(np.int64)
    Ymin = pd.to_numeric(test_df.iloc[i]['Bounding_boxes'].split(', ')[1]).astype(np.int64)
    Xmax = pd.to_numeric(test_df.iloc[i]['Bounding_boxes'].split(', ')[2]).astype(np.int64)
    Ymax = pd.to_numeric(test_df.iloc[i]['Bounding_boxes'].split(', ')[3]).astype(np.int64)
    #print('Xmin={}, Ymin={}, Xmax={}, Ymax={}'.format(Xmin, Ymin, Xmax, Ymax))
    test_data = ('test_image/' + test_df.iloc[i]['File_name']+','+str(Xmin)+','+str(Ymin)+ ','+str(Xmax)+','+str(Ymax)+','+str(test_df.iloc[i]['Coarse_lesion_type']))
    #print(type(val_data))
    print(test_data)
    #手動將print出的結果複製, 並存成csv檔

In [ ]:
#驗證 TensorFlow-GPU 安裝是否成功
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())